In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np


# Charger les données (assurez-vous que le fichier 'dvf_loyers.csv' est dans le même dossier)
# Si le fichier n'est pas trouvé, une erreur sera levée.
try:
    df = pd.read_csv('dvf_loyers.csv', sep=';')
except FileNotFoundError:
    print("Erreur: Le fichier 'dvf_loyers.csv' est introuvable.")
    print("Veuillez vous assurer que le fichier est dans le même répertoire que votre notebook.")
    # Crée un DataFrame vide pour éviter d'autres erreurs si le fichier n'est pas trouvé
    df = pd.DataFrame(columns=['valeur_fonciere', 'surface_reelle_bati', 'loyer_ref_m2', 'adresse_nom_voie', 'nom_commune', 'loyer_majore_m2', 'loyer_minore_m2'])

In [2]:
# Nettoyer les données
df = df.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 'loyer_ref_m2', 'loyer_majore_m2', 'loyer_minore_m2'])
df = df[df['surface_reelle_bati'] > 0]
df = df[df['loyer_ref_m2'] > 0]

In [3]:
# Créer la liste des biens pour le dropdown
df['bien_label'] = df.apply(
    lambda row: f"{row['adresse_nom_voie']}, {row['nom_commune']} - {row['valeur_fonciere']:,.0f}€ - {row['surface_reelle_bati']:.0f}m²",
    axis=1
)
quartiers = ["Tous"] + sorted(df["nom_commune"].unique().tolist())

In [4]:
# Widgets pour les paramètres
quartier_selector = widgets.Dropdown(
    options=quartiers,
    value="Tous",
    description="Quartier :",
    style={'description_width': '150px'},
    layout=widgets.Layout(width='90%')
)
bien_selector = widgets.Dropdown(
    options=list(zip(df['bien_label'], df.index)),
    description='Bien:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='90%')
)

apport_slider = widgets.IntSlider(
    value=50000,
    min=0,
    max=500000,  # Cette valeur max sera maintenant dynamique
    step=5000,
    description='Apport (€):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

taux_slider = widgets.FloatSlider(
    value=3.5,
    min=0.0,
    max=15.0,
    step=0.1,
    description='Taux prêt (%):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

duree_slider = widgets.IntSlider(
    value=25,
    min=5,
    max=30,
    step=1,
    description='Durée (années):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

In [5]:
def calculer_mensualite(capital, taux_annuel, duree_annees):
    """Calcule la mensualité d'un prêt"""
    if capital <= 0:
        return 0
    taux_mensuel = taux_annuel / 100 / 12
    nb_mois = duree_annees * 12
    if taux_mensuel == 0:
        return capital / nb_mois
    mensualite = capital * (taux_mensuel * (1 + taux_mensuel)**nb_mois) / ((1 + taux_mensuel)**nb_mois - 1)
    return mensualite

In [6]:
def mettre_a_jour_biens(change=None):
    """Met à jour la liste des biens selon le quartier sélectionné"""
    selected_quartier = quartier_selector.value
    
    if selected_quartier == "Tous":
        df_filtre = df
    else:
        df_filtre = df[df["nom_commune"] == selected_quartier]
    
    # Met à jour le dropdown des biens
    bien_selector.options = list(zip(df_filtre['bien_label'], df_filtre.index))
    
    # Sélectionne le premier bien du filtre si dispo
    if len(df_filtre) > 0:
        bien_selector.value = df_filtre.index[0]


In [7]:
def afficher_resultats(change=None):
    """Affiche les résultats de la simulation"""
    with output:
        clear_output(wait=True)
        
        # S'assurer qu'il y a des données à traiter
        if df.empty or bien_selector.value is None:
            print("Aucune donnée à afficher. Veuillez vérifier le chargement du fichier CSV.")
            return
            
        # Récupérer le bien sélectionné
        idx = bien_selector.value
        bien = df.loc[idx]
        
        # Paramètres de financement
        prix_achat = bien['valeur_fonciere']

        # --- MODIFICATION ---
        # 1. Mettre à jour la valeur maximale du curseur d'apport avec le prix du bien
        apport_slider.max = prix_achat
        
        # 2. Si l'apport actuel est supérieur au prix du bien, le ramener au maximum possible
        if apport_slider.value > prix_achat:
            apport_slider.value = prix_achat
        # --- FIN DE LA MODIFICATION ---
            
        apport = apport_slider.value
        taux = taux_slider.value
        duree = duree_slider.value
        
        # Calculs
        montant_emprunte = max(0, prix_achat - apport)
        mensualite_pret = calculer_mensualite(montant_emprunte, taux, duree)
        cout_total_credit = mensualite_pret * duree * 12
        cout_total_projet = cout_total_credit + apport
        
        # Calculs des loyers
        surface = bien['surface_reelle_bati']
        loyer_estime = surface * bien['loyer_ref_m2']
        loyer_majore = surface * bien['loyer_majore_m2']
        loyer_minore = surface * bien['loyer_minore_m2']
        
        # Charges estimées (25% du loyer)
        charges_estimees = loyer_estime * 0.25
        
        # Cash-flow
        cashflow_net = loyer_estime - mensualite_pret
        cashflow_max = loyer_majore - mensualite_pret
        
        # Rendements
        rendement_brut = (loyer_estime * 12 / prix_achat) * 100 if prix_achat > 0 else 0
        rendement_locatif_majore = (loyer_majore * 12 / prix_achat) * 100 if prix_achat > 0 else 0
        
        # Affichage formaté
        print("\n" + "-"*80)
        print("FINANCEMENT")
        print("-"*80)
        apport_pourcentage = (apport / prix_achat * 100) if prix_achat > 0 else 0
        print(f"Apport personnel:     {apport:>15,.0f} € ({apport_pourcentage:.1f}%)")
        print(f"Montant emprunté:     {montant_emprunte:>15,.0f} €")
        print(f"Mensualité du prêt:   {mensualite_pret:>15,.0f} €/mois")
        print(f"Coût total du crédit: {cout_total_credit:>15,.0f} €")
        print(f"Coût total du projet: {cout_total_projet:>15,.0f} € (Apport + Coût crédit)")


        print("\n" + "-"*80)
        print("LOYERS ET RENTABILITÉ")
        print("-"*80)
        print(f"Loyer MINORÉ:         {loyer_minore:>15,.0f} €/mois")
        print(f"Loyer de RÉFÉRENCE:   {loyer_estime:>15,.0f} €/mois")
        print(f"Loyer MAJORÉ:         {loyer_majore:>15,.0f} €/mois\n")

        print(f"Loyer annuel (réf.):  {loyer_estime*12:>15,.0f} €")
        print(f"Loyer/m² (réf.):      {bien['loyer_ref_m2']:>15.2f} €")
        print(f"Charges locatives:    {charges_estimees:>15,.0f} €/mois (estimé à 25%)")
        print(f"(refacturées au locataire)")
        
        cashflow_status_net = "POSITIF" if cashflow_net >= 0 else "NÉGATIF"
        print(f"\nCash-flow net (réf.): {cashflow_net:>15,.0f} €/mois ({cashflow_status_net})")
        cashflow_status_max = "POSITIF" if cashflow_max >= 0 else "NÉGATIF"
        print(f"Cash-flow max (majoré):{cashflow_max:>15,.0f} €/mois ({cashflow_status_max})")
        
        print(f"\nRendement brut (réf.):     {rendement_brut:>12.2f} %")
        print(f"Rendement brut (majoré):   {rendement_locatif_majore:>12.2f} %")


In [8]:
# Observer les changements
quartier_selector.observe(mettre_a_jour_biens, names='value')
bien_selector.observe(afficher_resultats, names='value')
apport_slider.observe(afficher_resultats, names='value')
taux_slider.observe(afficher_resultats, names='value')
duree_slider.observe(afficher_resultats, names='value')

In [9]:
# Affichage des widgets
display(quartier_selector)
display(bien_selector)
display(apport_slider)
display(taux_slider)
display(duree_slider)
display(output)

# Affichage initial
afficher_resultats()

Dropdown(description='Quartier :', layout=Layout(width='90%'), options=('Tous', 'Paris 10e Arrondissement', 'P…

Dropdown(description='Bien:', layout=Layout(width='90%'), options=(('PAS BASFOUR, Paris 2e Arrondissement - 17…

IntSlider(value=50000, description='Apport (€):', layout=Layout(width='80%'), max=500000, step=5000, style=Sli…

FloatSlider(value=3.5, description='Taux prêt (%):', layout=Layout(width='80%'), max=15.0, style=SliderStyle(d…

IntSlider(value=25, description='Durée (années):', layout=Layout(width='80%'), max=30, min=5, style=SliderStyl…

Output()

In [ ]:
#commentaire

In [ ]:
"""budget
taux
apport
duree
prix locatif"""

SyntaxError: invalid syntax (2167673405.py, line 5)